In [1]:
pip install -q recommenders faiss-gpu

Note: you may need to restart the kernel to use updated packages.


In [2]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
import numpy as np


def apk(actual, predicted, k=12):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=12):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [3]:
import gc
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import faiss
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
import datetime
from pathlib import Path

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) 
[GCC 9.4.0]
Pandas version: 1.3.5
Tensorflow version: 2.8.0


In [4]:
if os.getenv('LOCAL'):
    print('local')
    INPUT_DIR = Path('./input/transformed')
    OUTPUT_DIR = Path('./output/lightgcn_1')
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
else:
    print('kaggle')
    INPUT_DIR = Path('../input/transformed')
    OUTPUT_DIR = Path('/kaggle/working')

local


In [5]:
transactions = pd.read_pickle(INPUT_DIR / 'transactions_train.pkl')[['user', 'item', 't_dat']]
valid_start_date = datetime.date(2020, 9, 16)

In [6]:
def train_valid_stats(train, valid):
    train_users = set(train.user)
    train_items = set(train.item)
    valid_users = set(valid.user)
    valid_items = set(valid.item)
    print(f"train transaction: {len(train)}, train user: {len(train_users)}, train item: {len(train_items)}")
    print(f"valid transaction: {len(valid)}, valid user: {len(valid_users)}, valid item: {len(valid_items)}")
    print(f"valid user coverage: {len(train_users & valid_users) / len(valid_users)}")
    print(f"valid item coverage: {len(train_items & valid_items) / len(valid_items)}")

In [7]:
train_days = 90
recent_item_days = 7
recent_user_days = 21

In [8]:
print("train vaild split")
train_start_date = valid_start_date - datetime.timedelta(days=train_days)
train = transactions.query("@train_start_date <= t_dat < @valid_start_date").reset_index(drop=True)
valid = transactions.query("@valid_start_date <= t_dat").reset_index(drop=True)
del transactions
train_valid_stats(train, valid)

train vaild split
train transaction: 4102542, train user: 534006, train item: 42505
valid transaction: 240311, valid user: 68984, valid item: 17986
valid user coverage: 0.7323002435347327
valid item coverage: 0.9496274880462582


In [9]:
print("filter recent items")
train_item_start_date = valid_start_date - datetime.timedelta(days=recent_item_days)
recent_items = set(train.query("@train_item_start_date <= t_dat < @valid_start_date")['item'])
train = train.query("item in @recent_items").reset_index(drop=True)
train_valid_stats(train, valid)

filter recent items
train transaction: 3638425, train user: 523996, train item: 18611
valid transaction: 240311, valid user: 68984, valid item: 17986
valid user coverage: 0.725632030615795
valid item coverage: 0.775992438563327


In [10]:
print("filter each user recent transactions (train)")
train['last_t_dat'] = train.groupby('user').t_dat.transform(max)
train['diff_t_dat'] = (train.last_t_dat - train.t_dat).dt.days
train = train.query("diff_t_dat < @recent_user_days").reset_index(drop=True)
train_valid_stats(train, valid)

filter each user recent transactions (train)
train transaction: 2322691, train user: 523996, train item: 18611
valid transaction: 240311, valid user: 68984, valid item: 17986
valid user coverage: 0.725632030615795
valid item coverage: 0.775992438563327


In [11]:
print("drop duplicates (train)")
train = train[['user', 'item']]
valid = valid[['user', 'item']]
train = train.drop_duplicates(ignore_index=True)
train_valid_stats(train, valid)

drop duplicates (train)
train transaction: 2055546, train user: 523996, train item: 18611
valid transaction: 240311, valid user: 68984, valid item: 17986
valid user coverage: 0.725632030615795
valid item coverage: 0.775992438563327


In [12]:
print("drop users and items which apper only once (train)")
for _ in range(3):
    users = train.groupby('user').size().reset_index(name='sz').query("sz > 1").user
    train = train.query("user in @users")
    print(train.shape, len(train.user.unique()), len(train.item.unique()))

    items = train.groupby('item').size().reset_index(name='sz').query("sz > 1").item
    train = train.query("item in @items")
    print(train.shape, len(train.user.unique()), len(train.item.unique()))

train_valid_stats(train, valid)

drop users and items which apper only once (train)
(1931901, 2) 400351 18571
(1931198, 2) 400349 17868
(1931117, 2) 400268 17868
(1931111, 2) 400268 17862
(1931111, 2) 400268 17862
(1931111, 2) 400268 17862
train transaction: 1931111, train user: 400268, train item: 17862
valid transaction: 240311, valid user: 68984, valid item: 17986
valid user coverage: 0.5934709497854574
valid item coverage: 0.7618703436005783


In [13]:
# LightGCNが学習できるような形式にする
# - testのみに含まれるユーザーがあってはいけない
# - カラム名をあわせる
# - ratingカラムを追加
users = sorted(set(train.user))
valid = valid.query("user in @users")

train = train.rename(columns={'user': 'userID', 'item': 'itemID'})
valid = valid.rename(columns={'user': 'userID', 'item': 'itemID'})
train['rating'] = 1
valid['rating'] = 1

train.head()

,userID,itemID,rating
0,45,94260,1
1,45,97694,1
2,45,61400,1
3,62,93655,1
4,62,93300,1


In [14]:
data = ImplicitCF(train=train, test=valid, seed=42)

In [15]:
hparams = prepare_hparams(
    model_type='lightgcn',
    embed_size=128,
    n_layers=3,
    batch_size=8192,
    decay=0.0001,
    epochs=1000,
    learning_rate=0.001,
    eval_epoch=10,
    top_k=12,
    save_model=True,
    save_epoch=100,
    metrics=['recall', 'ndcg', 'precision', 'map'],
    MODEL_DIR=str(OUTPUT_DIR),
)
with Timer() as prepare_time:
    model = LightGCN(hparams, data, seed=42)
del data
print(f"{prepare_time.interval} sec")

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


2022-03-15 23:14:07.521098: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-15 23:14:07.556394: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 23:14:07.558479: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-15 23:14:07.558646: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

266.2070434089983 sec


In [16]:
gc.collect()
with Timer() as train_time:
    model.fit()
print(f"{train_time.interval} sec")

Epoch 1 (train)45.6s: train loss = 0.57713 = (mf)0.57704 + (embed)0.00009
Epoch 2 (train)43.9s: train loss = 0.27655 = (mf)0.27604 + (embed)0.00051
Epoch 3 (train)43.9s: train loss = 0.21791 = (mf)0.21719 + (embed)0.00072
Epoch 4 (train)43.5s: train loss = 0.18543 = (mf)0.18454 + (embed)0.00088
Epoch 5 (train)43.7s: train loss = 0.16235 = (mf)0.16132 + (embed)0.00103
Epoch 6 (train)43.8s: train loss = 0.14534 = (mf)0.14416 + (embed)0.00117
Epoch 7 (train)43.6s: train loss = 0.13088 = (mf)0.12958 + (embed)0.00130
Epoch 8 (train)43.8s: train loss = 0.11872 = (mf)0.11729 + (embed)0.00143
Epoch 9 (train)43.7s: train loss = 0.10930 = (mf)0.10775 + (embed)0.00155
Epoch 10 (train)43.8s + (eval)12.5s: train loss = 0.10066 = (mf)0.09900 + (embed)0.00166, recall = 0.01695, ndcg = 0.01108, precision = 0.00471, map = 0.00583
Epoch 11 (train)43.5s: train loss = 0.09351 = (mf)0.09174 + (embed)0.00177
Epoch 12 (train)44.0s: train loss = 0.08748 = (mf)0.08560 + (embed)0.00187
Epoch 13 (train)43.6s: tr

KeyboardInterrupt: 

In [17]:
# to avoid oom
tmp = valid[['userID']].drop_duplicates()
step = len(tmp) // 10
res = []
for i in range(0, len(tmp), step):
    res.append(model.recommend_k_items(tmp.iloc[i:i+step], top_k=12, remove_seen=False))
pred = pd.concat(res).reset_index(drop=True)
del tmp

In [18]:
pred = pred.rename(columns={'userID': 'user', 'itemID': 'item_pred'})
pred = pred.groupby('user')['item_pred'].apply(list).reset_index()
pred

,user,item_pred
0,86,"[27905, 33868, 102710, 103652, 103583, 27907, ..."
1,107,"[61303, 61296, 61305, 92029, 61297, 61304, 846..."
2,179,"[104986, 105180, 95216, 3510, 95217, 104072, 3..."
3,330,"[71106, 102240, 104840, 103129, 79488, 99399, ..."
4,349,"[99222, 87495, 2252, 87497, 85116, 95000, 2263..."
...,...,...
40935,1371737,"[75441, 97060, 84950, 58503, 98593, 92920, 101..."
40936,1371747,"[82631, 82628, 56694, 3091, 99766, 104157, 102..."
40937,1371813,"[38564, 28989, 99944, 38565, 38562, 86991, 367..."
40938,1371868,"[17428, 37189, 51179, 92457, 37185, 87911, 289..."


In [19]:
valid = pd.read_pickle(INPUT_DIR / 'transactions_train.pkl')[['user', 'item', 't_dat']].query("t_dat >= @valid_start_date")[['user', 'item']].rename(columns={'item': 'item_valid'}).reset_index(drop=True)
valid = valid.groupby('user')['item_valid'].apply(list).reset_index()
valid

,user,item_valid
0,80,[28967]
1,86,[87371]
2,107,"[69711, 77256, 33872]"
3,117,[97391]
4,179,"[102397, 98409, 73, 95784, 103796, 105103, 103..."
...,...,...
68979,1371868,"[97531, 103424]"
68980,1371871,"[94310, 91533]"
68981,1371879,"[2118, 84994, 57078, 84991, 101099, 104036, 87..."
68982,1371937,"[67261, 70640]"


In [20]:
merged = valid.merge(pred, on='user')
mpk = mapk(merged['item_valid'], merged['item_pred'])
users_valid = set(valid['user'])
users_merged = set(merged['user'])
user_coverage = len(users_valid & users_merged) / len(users_valid)
print(f"mapk: {mpk}, user coverage: {user_coverage}")

mapk: 0.026637331990289448, user coverage: 0.5934709497854574


In [21]:
pred.to_csv(OUTPUT_DIR / 'pred.csv', index=False)
model.infer_embedding(OUTPUT_DIR / 'user_emb.csv', OUTPUT_DIR / 'item_emb.csv')

users which are not included in pred

In [22]:
item_emb = pd.read_csv(OUTPUT_DIR / 'item_emb.csv', sep='\t', names=['item', 'emb'])
item_emb['emb'] = item_emb['emb'].apply(lambda x: list(map(float, x.split(' '))))

In [23]:
users_in_pred = pred['user'].values
items_with_emb = item_emb['item'].values

In [24]:
transactions = pd.read_pickle(INPUT_DIR / 'transactions_train.pkl')[['user', 'item', 't_dat']]
transactions = transactions.query("user not in @users_in_pred")
transactions = transactions.query("item in @items_with_emb")

pred2 = transactions.query("@train_start_date <= t_dat < @valid_start_date").reset_index(drop=True)

pred2['sz'] = pred2.groupby(['user', 'item'])['item'].transform('size')
pred2 = pred2.sort_values(by=['user', 't_dat', 'sz'], ascending=False)

pred2 = pred2[['user', 'item']].drop_duplicates(ignore_index=True)

pred2 = pred2.groupby('user')['item'].apply(list).reset_index()
pred2['item'] = pred2['item'].apply(lambda x: x[0])
pred2

,user,item
0,0,16023
1,1,87145
2,2,78503
3,4,101367
4,6,58295
...,...,...
485782,1371969,82965
485783,1371975,56446
485784,1371976,71110
485785,1371977,71107


In [25]:
pred2 = pred2.merge(item_emb, on='item')
pred2

,user,item,emb
0,0,16023,"[-0.13813072, 0.4008881, -0.50532943, -1.35958..."
1,3768,16023,"[-0.13813072, 0.4008881, -0.50532943, -1.35958..."
2,5087,16023,"[-0.13813072, 0.4008881, -0.50532943, -1.35958..."
3,13619,16023,"[-0.13813072, 0.4008881, -0.50532943, -1.35958..."
4,23672,16023,"[-0.13813072, 0.4008881, -0.50532943, -1.35958..."
...,...,...,...
485782,1369176,102014,"[0.17385736, -0.05746226, 0.20658967, -0.12867..."
485783,1369366,55741,"[-0.39888167, -0.14153816, 0.29024222, -0.3110..."
485784,1370569,89908,"[0.2322186, 0.3250174, 0.0035811216, -0.001440..."
485785,1371036,92443,"[0.1843273, -0.600322, -0.054204337, -0.000332..."


In [26]:
emb = np.array(item_emb['emb'].values.tolist(), dtype=np.float32)
pred2_emb = np.array(pred2['emb'].values.tolist(), dtype=np.float32)

In [27]:
index = faiss.index_factory(128, "Flat", faiss.METRIC_INNER_PRODUCT)
index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, index)
index.add(emb)
_, nns = index.search(pred2_emb, 12)

In [28]:
mp = item_emb['item'].values
pred2['item_pred'] = [mp[nn].tolist() for nn in nns]

In [29]:
assert len(set(pred.user) & set(pred2.user)) == 0
pred_merged = pd.concat([pred, pred2[['user', 'item_pred']]]).reset_index(drop=True)

In [30]:
merged = valid.merge(pred_merged, on='user')
mpk = mapk(merged['item_valid'], merged['item_pred'])
users_valid = set(valid['user'])
users_merged = set(merged['user'])
user_coverage = len(users_valid & users_merged) / len(users_valid)
print(f"mapk: {mpk}, user coverage: {user_coverage}")

mapk: 0.0282399032702659, user coverage: 0.7272555955004059


users without transactions

In [31]:
start_date = valid_start_date - datetime.timedelta(days=7)
end_date = valid_start_date - datetime.timedelta(days=1)
transactions = pd.read_pickle(INPUT_DIR / 'transactions_train.pkl')[['t_dat', 'user', 'item']].query("@start_date <= t_dat <= @end_date")
users = pd.read_pickle(INPUT_DIR / 'users.pkl')[['user', 'age']]

transactions_age = transactions.query("@start_date <= t_dat <= @end_date").merge(users, on='user')
transactions_age['age'] = transactions_age['age'].fillna(0).astype(int)
age_volume = transactions_age.groupby('age').size().reset_index(name='sz').sort_values(by='age')
sz_threshold = age_volume.query("24 <= age <= 26").sz.sum()  # 最もtransactionsのボリュームが大きい25歳で幅1を取ったときの数を基準とする

# 各ageに対して閾値を超える最小の幅を求める
age_width = {}
for age in range(10, 100):
    for d in range(1, 100):
        lb = age - d
        ub = age + d
        sz = age_volume.query("@lb <= age <= @ub").sz.sum()
        if sz >= sz_threshold:
            age_width[age] = d
            break

age_items = {}
for age in range(10, 100):
    lb = age - age_width[age]
    ub = age + age_width[age]
    age_items[age] = transactions_age.query("@lb <= age <= @ub").groupby('item').size().reset_index(
        name='sz').sort_values(by='sz', ascending=False)['item'].values[:12].tolist()

# age: nanには全体の人気商品を割り当てる
age_items[0] = transactions_age.groupby('item').size().reset_index(
    name='sz').sort_values(by='sz', ascending=False)['item'].values[:12].tolist()

age_pred = pd.DataFrame([{'age': k, 'item': v} for k, v in age_items.items()])

In [32]:
pred3 = valid[['user']].merge(users, on='user')
pred3['age'] = pred3['age'].fillna(0).astype(int)
pred3 = pred3.merge(age_pred, on='age')[['user', 'item']].rename(columns={'item': 'item_pred'}).reset_index(drop=True)

In [33]:
users = set(pred.user.values) | set(pred2.user.values)
pred3 = pred3.query("user not in @users")
pred_merged = pd.concat([pred, pred2, pred3]).reset_index(drop=True)

In [34]:
merged = valid.merge(pred_merged, on='user')
mpk = mapk(merged['item_valid'], merged['item_pred'])
users_valid = set(valid['user'])
users_merged = set(merged['user'])
user_coverage = len(users_valid & users_merged) / len(users_valid)
print(f"mapk: {mpk}, user coverage: {user_coverage}")

mapk: 0.022987078438599047, user coverage: 1.0
